# 12th Day of Python 4 DIP

> Contents:
> 1. Morphologocial filters
> 2. Fourier Transform and DFT

 

Execute the cells to gain outputs

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skimage as SK
from Tools import *
%matplotlib inline

### Removing small objects with morphology

In [ ]:
Circles = SK.color.rgb2gray(SK.io.imread(("./Figures/circles.jpg")))
ShowTime(Circles,With_Hist=False,colorbar=False,Im_title="Circles")

In [ ]:
Circles[Circles>=0.5] = 1
Circles[~(Circles > 0.5)] = 0
Circles = np.int64(Circles).astype(bool)
fig,ax = plt.subplots(nrows=2,ncols=2,figsize=(8,8))
ax = ax.ravel()
ax[0].imshow(Circles,cmap='gray')
ax[0].set_title("Original")
ax[0].axis("off")
for i,j in enumerate([25,100,400]):
    Out = SK.morphology.remove_small_objects(Circles,min_size=j,connectivity=1)
    ax[i+1].imshow(Out,cmap='gray')
    ax[i+1].axis("off")
    ax[i+1].set_title("Removed with %d radius."%j)


### Boundary detection

In [ ]:
Horse = SK.data.horse()
Bound_Horse = Horse ^ SK.morphology.binary_erosion(Horse)
ShowTimeMul((Horse,Bound_Horse),With_Hist=False,colorbar=False,Im_title=["Original","Boundary"],cmaps=["gray"]*2)

### Fourier Transform and DFT

In [ ]:
Camera = SK.io.imread("./Figures/cameraman.jpg",as_gray=True)
Camera_noised = SK.util.random_noise(Camera,mode='gaussian',seed=1000)
PSNR = SK.metrics.peak_signal_noise_ratio(Camera,Camera_noised)
print(f"The PSNR of Noised image is {PSNR}")
ShowTimeMul((Camera,Camera_noised),With_Hist=False,colorbar=False,Im_title=["Original","Noised"],cmaps=["gray"]*2)

##### Plotting Frequency Spectrum

In [ ]:
Freq = np.fft.fft2(Camera)
FFT = np.fft.fftshift(Freq)
Camera_FFT = 5*np.log10(0.1+FFT).real
FFT_Phase = np.angle(Camera_FFT)
Camera_IFFT = np.fft.ifft2(Freq).real
ShowTimeMul((Camera,Camera_FFT),With_Hist=False,colorbar=False,Im_title=["Image","FFT"],cmaps=["gray"]*2)
ShowTimeMul((FFT_Phase,np.clip(Camera_IFFT,0,255)),With_Hist=False,colorbar=False,Im_title=["Phase","IFFT"],cmaps=["gray"]*2)

#### Gaussian Kernel as Low Pass Filter

In [ ]:
Gauss_Ker = gauss(50,2.0)
Freq_Gauss = np.fft.fft2(np.fft.ifftshift(Gauss_Ker)).real
Freq_Gauss2 = 5*np.log10(0.1+np.fft.fftshift(Freq_Gauss)).real
ShowTimeMul((Gauss_Ker,Freq_Gauss2),Im_title=["Kernel","Shifted Freq"],colorbar=True,cmaps=["gray"]*2,With_Hist=False)

### Spectrum of Filtered Image by Gauss

In [ ]:
Camera_Blurred = FastConv2D(Camera,gauss(25,3.0))
Cam_Blur_FFT = 5*np.log10(0.1+np.fft.fftshift(np.fft.fft2(Camera_Blurred))).real
ShowTimeMul((Camera_FFT,Cam_Blur_FFT),Im_title=["Original Spectrum","Blurred Spectrum"],colorbar=False,cmaps=["gray"]*2,With_Hist=False)

### High pass filtering

In [ ]:
Freq_Cam = np.fft.fft2(Camera)
M,N = Freq_Cam.shape
Cx,Cy = M//2,N//2
fig,ax = plt.subplots(nrows=4,ncols=4,figsize=(8,8))
ax = ax.ravel()

for i in range(1,17):
    Freq = Freq_Cam.copy()
    Freq = np.fft.fftshift(Freq)
    Freq[Cx-i:Cx+i+1,Cy-i:Cy+i+1] = 1
    Image = np.clip(np.fft.ifft2(np.fft.ifftshift(Freq)).real,0,255)
    ax[i-1].imshow(Image,cmap="gray")
    ax[i-1].set_title("Filter %d." %i)
    ax[i-1].axis("off")